# Установка Evidently

In [ ]:
!pip install evidently

# Подготовка данных

Для нашего примера мы будем использовать датасет о ценах на недвижимость. Предположим, у нас есть два датафрейма: reference_data (данные, на которых обучалась модель) и current_data (текущие данные).

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

# Загрузка данных
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['target'] = housing.target

# Разделение на датасеты reference и current
reference_data = df.iloc[:10000]
current_data = df.iloc[10000:20000]

# Обнаружение Data Drift
Теперь используем Evidently для обнаружения Data Drift:

In [ ]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import DatasetDriftMetric

# Создание отчёта
report = Report(metrics=[
    DataDriftPreset(),
])

# Выполнение анализа
report.run(reference_data=reference_data, current_data=current_data)

# Сохранение отчёта в HTML
report.save_html("data_drift_report.html")

# Анализ предсказаний модели
Допустим, у нас есть модель, которая предсказывает цены на недвижимость. Мы можем использовать Evidently для мониторинга предсказаний:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from evidently.report import Report
from evidently.metric_preset import TargetDriftPreset

# Обучение модели
model = RandomForestRegressor()
model.fit(reference_data.drop('target', axis=1), reference_data['target'])

# Получение предсказаний
reference_data['prediction'] = model.predict(reference_data.drop('target', axis=1))
current_data['prediction'] = model.predict(current_data.drop('target', axis=1))

# Создание отчёта
report = Report(metrics=[
    TargetDriftPreset(),
])

report.run(reference_data=reference_data, current_data=current_data)
report.save_html("model_performance_report.html")

# Мониторинг признаков
Evidently также позволяет детально анализировать изменения в отдельных признаках:

In [ ]:
from evidently.metric_preset import DataQualityPreset

# Создание отчёта по качеству данных
data_quality_report = Report(metrics=[
    DataQualityPreset(),
])

data_quality_report.run(reference_data=reference_data, current_data=current_data)
data_quality_report.save_html("data_quality_report.html")

# Настройка алертов
Evidently позволяет настраивать алерты на основе пороговых значений:

In [ ]:
from evidently.options import ColorOptions
from evidently.test_suite import TestSuite
from evidently.tests import TestColumnDrift

test_suite = TestSuite(tests=[
    TestColumnDrift(column_name="MedInc", stattest_threshold=0.2),
    TestColumnDrift(column_name="HouseAge", stattest_threshold=0.1, stattest='ks'),
])

test_suite.run(reference_data=reference_data, current_data=current_data)
test_suite.save_html("drift_test_suite.html")